# À la façon d'Andy Warhol - 3

L'activité suivante complète les activités de seconde et première situées dans le même dossier.

Il peut être nécessaire de revoir ces activités avant d'aborder celle-ci.

On commence par importer toutes les bibliothèques nécessaires :

In [ ]:
# installation de pillow
import sys
!{sys.executable} -m pip install pillow
# puis importation du module Image
from PIL import Image
# utilisation de widgets pour une disposition en ligne des images
import ipywidgets as widgets
# utilisation de io pour conversion des images
import io
# utilisation de chemin normalisé vers les fichiers du système
import os.path as path

 La classe `Image` du module `Image` contient [une méthode `split`](https://pillow.readthedocs.io/en/stable/reference/Image.html#PIL.Image.Image.split) qui renvoie un tuple d'images correspondant aux canaux la composant :
 
 - une image de type `RGB` est composée de 3 canaux (Rouge, Vert et Bleu)
 - une image de type `RGBA` est composée de 4 canaux (Rouge, Vert, Bleu et Transparence)
 - une image `L` est composée d'un seul canal (niveau de gris)
 - etc.
 
[Voir les différents types](https://pillow.readthedocs.io/en/stable/handbook/concepts.html#concept-modes)

In [ ]:
fichier_image = path.join('images', 'boole.jpg')
img = Image.open(fichier_image)

img.split()

La méthode `split`, n'exprimant qu'un seul canal, renvoie une image de type `L`.

In [ ]:
position = (0, 0)

print(f"Le pixel haut gauche de l'image de départ contient {img.getpixel(position)}")

for i, couleur in enumerate(['rouge', 'vert', 'bleu']):
    print(f"Le pixel haut gauche du canal {couleur} de \
l'image de départ contient {img.split()[i].getpixel(position)}")

On peut alors utiliser la méthode des `widgets` vue précédemment qu'on transforme en fonctions renvoyant la boite à afficher :

In [ ]:
def conversion_widget(image, format="jpeg"):
    """renvoie un widget IPython/Jupyter contenant l'image à afficher"""
    img_byte_arr = io.BytesIO()
    image.save(img_byte_arr, format=format)
    img_byte_arr = img_byte_arr.getvalue()
    return widgets.Image(value=img_byte_arr)

def boiteNB_images_rvb(image):
    """renvoie un widget HBox contenant les images NB issues des canaux de l'image"""
    wid = []
    for im in image.split():
        wid.append(conversion_widget(im))
    return widgets.HBox(wid)
    
display(boiteNB_images_rvb(img))

On peut constater que les images sont en noir et blanc (enfin plutôt en nuances de gris) et ne sont pas les mêmes puisque chacune est l'expression d'une couleur différente. Pour permettre l'identification des images par leur couleur d'origine, on peut les coloriser grâce à la fonction de classe [merge](https://pillow.readthedocs.io/en/stable/reference/Image.html#PIL.Image.merge).

Pour ne pas exprimer les deux autres canaux, on crée une image vide (avec des 0) ayant la même dimension.

In [ ]:
def coloriser_image_L(L_image, numero_canal):
    image_vide = Image.new("L", L_image.size, "black")
    canaux = [image_vide, image_vide, image_vide]
    # on exprime le canal souhaité
    canaux[numero_canal] = L_image
    return Image.merge("RGB", tuple(canaux))


def boite_images_rvb(image):
    """renvoie un widget HBox contenant les images couleurs issues des canaux de l'image"""
    wid = []
    for n_canal, im in enumerate(image.split()):
        im_colorise = coloriser_image_L(im, n_canal)
        wid.append(conversion_widget(im_colorise))
    return widgets.HBox(wid)
    
display(boite_images_rvb(img))

On constate que mettre du rouge sur l'image NB exprimant le rouge est tout à fait artificiel : on aurait bien pu la coloriser dans une autre couleur.

Certains satellites produisent des images sur des bandes invisibles à l'oeil, c'est la cas des satellites européens [Pléiades](https://fr.wikipedia.org/wiki/Pl%C3%A9iades_(satellite)) qui intègre un capteur **infra-rouge** :

![](infrarouge.png)

Pour rendre compte *visiblement* de ce canal, les ingénieurs peuvent voir simplement une image NB ou alors ils rréorganisent les canaux des images en remplaçant le canal rouge par le canal infra-rouge : ainsi le rouge exprime l'infra-rouge.

C'est le cas dans cette image :

![](remplacement.png)


[issue de ce site](https://e-cours.univ-paris1.fr/modules/uved/envcal/html/compositions-colorees/3-differentes-composition-coloree/2-1-vraie-fausse-couleur%20.html).

L'infra-rouge permet notamment de détecter sur la Terre les zones humides et de différencier les natures de sols. Ce canal est aussi utilisé en astro-photographie. Les photographes amateurs de ce genre de photographie modifie leur appareil (à leurs risques et périls) pour pouvoir enregister ce canal. 

On écrit alors à partir de la fonction `coloriser_image` une fonction`fusion_canaux` qui fusionne les canaux demandés :

In [ ]:
def fusion_canaux(image, liste_canaux):
    canaux = image.split()
    image_vide = Image.new("L", image.size, "black")
    canaux_voulus = [image_vide, image_vide, image_vide]
    for canal in liste_canaux:
        canaux_voulus[canal] = canaux[canal]
    return Image.merge("RGB", tuple(canaux_voulus))

display(fusion_canaux(img, [1]))
display(fusion_canaux(img, [0, 1]))

Sur le modèle de la fonction `boite_image_rvb`, on définit la fonction `boite_image` :

In [ ]:
def boite_images(image, liste_canaux):
    """renvoie un widget HBox contenant les images couleurs issues des canaux de l'image"""
    wid = []
    
    for liste in liste_canaux:
        im = fusion_canaux(image, liste)
        wid.append(conversion_widget(im))
    return widgets.HBox(wid)

display(boite_images(img, [[0], [1], [2]]))

On termine avec un affichage d'images en 3 x 3 (3 en largeur et 3 en hauteur).

Sur le même pricipe vue avec le widget `HBox`, on peut empiler verticalement avec un widget `VBox`.

In [ ]:
def matrice_images(image, liste_canaux):
    wid = []
    
    liste_canaux = [liste_canaux[i:i + 3] for i in range(0, len(liste_canaux), 3)]
    for liste in liste_canaux:
        wid.append(boite_images(image, liste))
    return widgets.VBox(wid)
    
canaux_voulus = [[0], [1], [0, 1], [1], [2], [1, 2], [2], [0], [2, 0]]
display(matrice_images(img, canaux_voulus))